In [54]:
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, LpMinimize, GLPK, lpSum



__Maximize__ $$–4x_1 + 2x_2$$
__Subject To__
$$4x_1 + x_2 + x_3 = 20$$
$$2x_1 – x_2 ≥ 6$$
$$x_1 – x_2 + 5x_3 ≥ –5$$
$$–3x_1 + 2x_2 + x_3 ≤ 4$$ 

$$x_1 ≤ 0, x_2 ≥ 0, x_3\text{ unrestricted}$$


In [133]:
# Creates a list of the Ingredients
periods = list(range(1,6))
products = ['widgets', 'gadgets', 'flugels']
ad_costs_list = [160, 120, 180]
product_selling_price_list = [2490, 1990, 2970]

ad_costs = dict(zip(products, ad_costs_list))
product_selling_prices = dict(zip(products, product_selling_price_list))

In [135]:
# Final
ads_variables = []
demand_from_contracts_variables = []
demand_from_ads_variables = []
# define variables

for period in periods:
    for product in products:
        demand_from_contracts_variables.append(f"{product}_contracted_demand_in_p{period}")
        if period == 5:
            pass
        else:
            ads_variables.append(f"{product}_advertising_investment_p{period}")
        if period == 1:
            pass
        else:
            demand_from_ads_variables.append(f"{product}_demand_from_ads_p{period}")

In [136]:
demand_variables

['widgets_total_demand_in_p1',
 'gadgets_total_demand_in_p1',
 'flugels_total_demand_in_p1',
 'widgets_total_demand_in_p2',
 'widgets_total_demand_in_p1',
 'gadgets_total_demand_in_p1',
 'flugels_total_demand_in_p1',
 'widgets_total_demand_in_p2',
 'gadgets_total_demand_in_p2',
 'flugels_total_demand_in_p2',
 'widgets_total_demand_in_p3',
 'gadgets_total_demand_in_p3',
 'flugels_total_demand_in_p3',
 'widgets_total_demand_in_p4',
 'gadgets_total_demand_in_p4',
 'flugels_total_demand_in_p4',
 'widgets_total_demand_in_p5',
 'gadgets_total_demand_in_p5',
 'flugels_total_demand_in_p5']

In [137]:
# A dictionary called 'ingredient_vars' is created to contain the referenced Variables
advertising_vars = LpVariable.dicts("", ads_variables, 0)
demand_from_contracts_vars = LpVariable.dicts("", demand_from_contracts_variables, 0)
demand_from_ads_vars = LpVariable.dicts("", demand_from_ads_variables, 0)
    
    
# defines the problem
nu_industries = LpProblem("NU Industries", LpMaximize)

Throughout the planning horizon, NU Industries will sell: 
* Widgets for \\$2490 
* Gadgets for \\$1990 
* Flugels for \\$2970. 

The products can be manufactured at either of NU's Manufacturing Plants.


In [138]:
nu_industries += (
    lpSum([product_selling_prices[product] * demand_from_ads_vars[f'{product}_demand_from_ads_p{period}'] for product 
           in products for period in periods if period != 1]) +  
    lpSum([product_selling_prices[product] * demand_from_contracts_vars[f'{product}_contracted_demand_in_p{period}'] for product 
           in products for period in periods]) -
    lpSum([ad_costs[product] * advertising_vars[f'{product}_advertising_investment_p{period}'] for product 
           in products for period in periods if period != 5]),
    'Maximize Profits'
)

# Define Constraints

Production Requirements:    
This is demand in a given period that we are already contracted to sell

|Product|Period 1|Period 2|Period 3|Period 4|Period 5|
|-------|--------|--------|--------|--------|--------|
|Widgets|70      |125     |185     |190     |200     |
|Gadgets|200     |300     |295     |245     |240     |
|Flugels|140     |175     |205     |235     |230     |

In [139]:
# Define Production Requirement Dictionary
production_contractual_requirements_list = [70, 200, 140, 125, 300, 175, 185, 295, 205, 190, 245, 235, 200, 240, 230]
production_contractual_requirements = dict(zip([f"p_{period}_{product}_requirement" for period 
                                                in periods for product in products], 
                                               production_contractual_requirements_list))


In [140]:
# Production Requirements - total demand must exceed our contracted production requirements
for period in periods:
    for product in products:
        nu_industries += (demand_from_contracts_vars[f'{product}_contracted_demand_in_p{period}']
                          >= production_contractual_requirements[f'p_{period}_{product}_requirement'], 
                          f"{product}_required_in_p{period}")


Marketing & Forecasting Department anticipates that NU Industries can cultivate additional demand within the distribution area through effective advertising. The M&F Department projects that each \\$160 invested in advertising Widgets in a particular period creates additional demand for one Widget in the next period. The corresponding values for Gadgets and Flugels are \\$120 and \\$180, respectively. However, the total advertising budget is limited to \\$70,000 for the entire planning horizon.

In [141]:
# Advertising Budget
nu_industries += (lpSum([advertising_vars[f'{product}_advertising_investment_p{period}'] for product 
                         in products for period in periods if period != 5]) <= 70000,
                  "Advertising_Budget")

In [146]:
# Additional Demand from Advertising
for period in periods:
    for product in products:
        if period == 1:
            pass
        else:
            nu_industries += (demand_from_ads_vars[f'{product}_demand_from_ads_p{period}'] * ad_costs[product] == 
                              advertising_vars[f'{product}_advertising_investment_p{period-1}'],
                              f"{product}_Demand_from_Ads_in_p{period}")

In [147]:
# The problem data is written to an .lp file
nu_industries.writeLP("NU_Industries.lp")

[_flugels_advertising_investment_p1,
 _flugels_advertising_investment_p2,
 _flugels_advertising_investment_p3,
 _flugels_advertising_investment_p4,
 _flugels_contracted_demand_in_p1,
 _flugels_contracted_demand_in_p2,
 _flugels_contracted_demand_in_p3,
 _flugels_contracted_demand_in_p4,
 _flugels_contracted_demand_in_p5,
 _flugels_demand_from_ads_p2,
 _flugels_demand_from_ads_p3,
 _flugels_demand_from_ads_p4,
 _flugels_demand_from_ads_p5,
 _gadgets_advertising_investment_p1,
 _gadgets_advertising_investment_p2,
 _gadgets_advertising_investment_p3,
 _gadgets_advertising_investment_p4,
 _gadgets_contracted_demand_in_p1,
 _gadgets_contracted_demand_in_p2,
 _gadgets_contracted_demand_in_p3,
 _gadgets_contracted_demand_in_p4,
 _gadgets_contracted_demand_in_p5,
 _gadgets_demand_from_ads_p2,
 _gadgets_demand_from_ads_p3,
 _gadgets_demand_from_ads_p4,
 _gadgets_demand_from_ads_p5,
 _widgets_advertising_investment_p1,
 _widgets_advertising_investment_p2,
 _widgets_advertising_investment_p3,
 _w